# IN4080: obligatory assignment 4
 
The final mandatory assignment for IN4080 consists of two parts. The first is about the development of dialogue systems, and the second about machine translation.
You are required to get at least 12/20 points to pass. 

- We assume that you have read and are familiar with IFI’s requirements and guidelines for mandatory assignments, see [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-mandatory.html) and [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-guidelines.html).
- This is an individual assignment. You should not deliver joint submissions. 
- You may redeliver in Devilry before the deadline (__Sunday, November 3 at 23:59__).
- Only the last delivery will be read! If you deliver more than one file, put them into a zip-archive. You don't have to include in your delivery the data files already provided for this assignment. 
- Name your submission _your\_username\_in4080\_mandatory\_4_

Part 1 should be done on your local computer, as it relies on a speech interface that will not work on remote machines. For Part 2, using _Fox_ is preferable, at least for the fine-tuning task.

You should deliver a completed version of this Jupyter notebook, containing both your code and explanations about the steps you followed. We want to stress that simply submitting code is __not__ by itself sufficient to complete the assignment - we expect the notebook to also contain explanations of what you have implemented, along with motivations for the choices you made along the way. Preferably use whole sentences, and mathematical formulas if necessary. Explaining in your own words (using concepts we have covered through in the lectures) what you have implemented and reflecting on your solution is an important part of the learning process - take it seriously!

Regarding the use of LLMs (ChatGPT or similar): you are allowed to use them as 'sparring partner', for instance to clarify something you have not understood. However, you are __not__ allowed to use them to generate solutions (either in part or in full) to the assignment tasks. 


## Part 1 : Dialogue systems

Our objective in this part is to build a spoken conversational interface for a (simulated) elevator. 

### Basic setup

First, let's make sure that we have all the necessary Python modules:

In [2]:
%pip install ipywidgets pyaudio openai-whisper pyttsx3 setfit spacy jellyfish
!python -m spacy download en_core_web_sm

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached openai_whisper-20240930-py3-none-any.whl
  Using cached pyttsx3-2.98-py3-none-any.whl.metadata (3.8 kB)
  Using cached setfit-1.1.0-py3-none-any.whl.metadata (12 kB)
  Using cached spacy-3.8.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (27 kB)
  Using cached jellyfish-1.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.6 kB)
  Using cached numba-0.60.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.7 kB)
  Using cached more_itertools-10.5.0-py3-none-any.whl.metadata (36 kB)
  Using cached tiktoken-0.8.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached pyobjc-10.3.1-py3-none-any.whl.metadata (26 kB)
  Using cached datasets-3.0.2-py3-none-any.whl.metadata (20 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.w

The code for the simulated elevator is provided below. The elevator is displayed using simple widgets (where the current floor is shown in green). 

In [3]:
import ipywidgets as widgets
from IPython.display import display

import time, random, string, threading
from typing import List, Tuple, Dict, Set

class BasicElevator:
    """Elevator simulated using a GUI"""
    
    def __init__(self, start_floor:int =1, nb_floors=10):
        """Initialised a new elevator, placed on the first floor"""
        
        # Current floor of the elevator
        self.cur_floor: int = start_floor

        # (Possibly empty) list of next floor stops to reach 
        self.next_stops : List[int] = []
        
        # Building the basic GUI showing the elevator
        display(self._build_gui(nb_floors))

        # Starts the thread executing the movements
        thread = threading.Thread(target=self.elevator_move_thread)
        thread.start()     
            
    def move_to_floor(self, floor_number : int):
        """Move to a given floor (by adding it to a stack of floors to reach)"""
        
        if floor_number < 1 or floor_number > len(self.floors):
            raise RuntimeError("Floor number must be between 1 and %i"%len(self.floors))

        self.next_stops.append(floor_number)
        
        
    def stop(self):
        """Stops all movements of the elevator"""
        self.next_stops.clear()

    def _build_gui(self, nb_floors):
        """Creates the GUI for the elevator, with a status label and a visual representation
        of the floors, where the current floor is indicated in green."""

        # Displaying the current status of the elevator (still or going up or down)
        status_label = widgets.HTML("<b>Status</b>: ")
        self.status = widgets.Label("Still")
        status_box = widgets.HBox([status_label, self.status])

        # Displaying the floors on a vertical axis
        self.floors = []
        floor_layout = widgets.Layout(width='50px', height='30px', border='2px solid black',justify_content="center")
        for i in range(1, nb_floors+1):
            floor = widgets.Label(value=str(i), layout=floor_layout)
            floor.style = {"background":("white" if i!=self.cur_floor else "lightgreen")}
            self.floors.append(floor)

        # Create a vertical box container to hold the boxes
        vbox = widgets.VBox([status_box] + self.floors[::-1])
        return vbox
    

    def elevator_move_thread(self, speed=1.0, latency=0.1):
        """Trigger a movement of the elevator if the list of next stops is not 
        empty. The movement continues until all goals are reached."""

        while True:
            while self.next_stops:
                if self.cur_floor == self.next_stops[0]:
                    del self.next_stops[0]
                    continue
                if self.cur_floor < self.next_stops[0]:
                    next_floor = self.cur_floor+1
                    self.status.value = "UP"
                elif self.cur_floor > self.next_stops[0]:
                    next_floor = self.cur_floor-1
                    self.status.value = "DOWN"
                time.sleep(speed)   
                self.floors[self.cur_floor-1].style.background = "white"
                self.floors[next_floor-1].style.background = "lightgreen"
                self.cur_floor = next_floor
            self.status.value = "Still"
            
            # Wait loop (until we have a goal in self.next_stops)
            time.sleep(latency)

The elevator can be easily controlled through the functions `move_to_floor` and `stop`:

In [4]:
elevator = BasicElevator()
elevator.move_to_floor(5)


We will now make our elevator controllable through a speech interface instead of using function calls.

## Speech interface

First, make sure that you have installed `pyaudio` (for audio processing), `whisper` (for speech recognition), and `pyttsx3` (for speech synthesis).

The `TalkingElevator` class below extends the basic simulated elevator with speech input and output. 

Upon clicking on the recording button, speech is recorded from the user's microphone, and continues until the stop button is clicked. The speech recognition engine `Whisper` from OpenAI is then employed to transcribe the spoken input (either on GPU, if you have a GPU on your machine, or on CPU). The transcription result is then sent to the `process_input` function, which is responsible for determining the system response. 

We are going to focus on implementing this `process_input` method. Note this system reaction to new user inputs may comprise both verbal responses (to be uttered by the system through the `_say_to_user` method) and physical actions (through the `move_to_floor` and `stop` methods).

In [5]:
import threading, time
import numpy as np
from typing import List, Tuple, Dict, Set
import whisper, pyaudio, pyttsx3
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

class TalkingElevator(BasicElevator):
    """Extension of the simulated elevator with a speech interface"""
    
    def __init__(self):
        print("Loading TTS and ASR models", end="...", flush=True)
        self.tts_engine = pyttsx3.init()  
        self.asr_engine = whisper.load_model("small.en")
        print("Done")
        
        # Initializing the GUI
        BasicElevator.__init__(self)

        # Starts the dialogue
        self.dialogue_history = []
        self._say_to_user("Hi, what can I do for you today?")
    

    def process_input(self, user_input: str, conf_score:float=1.0):
        """Processes the (transcribed) user input, and respond appropriately 
        (through a verbal response and possibly also an action, such as moving floors)"""

        self._add_to_dialogue_history(user_input, speaker="user", conf_score=conf_score)

        # Dummy response. Should be replaced by the actual dialogue behaviour
        self._say_to_user("Sorry, I don't understand you, pal")

    
    def _say_to_user(self, system_response: str):
        """Say something back to the user, and add the dialogue turn to the history. The 
        synthesis is done using the pyttsx3 library."""

        self._add_to_dialogue_history(system_response, speaker="elevator")
        
        # Stopping current TTS if one is active
        try:
            self.tts_engine.endLoop()
        except:
            pass
        self.tts_engine.say(system_response)
        self.tts_engine.runAndWait()


    def _add_to_dialogue_history(self, turn:str , speaker:str, conf_score:float=1.0):
        """Adds a new (user or system) turn to the dialogue history list, and displays it
         on the chat window displaying the turns"""

        self.dialogue_history.append({"speaker":speaker, "text":turn, 
                                      "conf_score":conf_score, "timesamp":time.time()})
        
        self.history_area.value += "&nbsp;<strong>%s</strong>:  %s"%(speaker.title(), turn)
        if conf_score < 1.0:
            self.history_area.value += " (%.2f)"%(conf_score)
        self.history_area.value += "<br>"
   
   
    def _build_gui(self, nb_floors):
        """GUI for the Talking elevator, comprising (beyond the simulated elevator from 
        BasicElevator) a chat window showing the dialogue turns, and buttons to record
        the user input. 
        The user should first click on the record button, then on stop when they have finished.
        Once the stop button is clicked, the audio is transcribed by Whisper, and finally 
        forwarded to the process_input function."""

        core_gui = BasicElevator._build_gui(self, nb_floors)

        self.frames = []
        self.recording = False

        def record(chunk_size=1024):
            """Record audio chunks to a buffer."""
            self.recording = True
            p = pyaudio.PyAudio()
            stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, 
                            frames_per_buffer=chunk_size)
            while self.recording:
                self.frames.append(stream.read(chunk_size))
            stream.close()

        def on_record_button_clicked(b):
            "Starts the recording"
            record_button.disabled=True
            stop_button.disabled=False
            self.frames = []  # Clear previous recordings
            thread = threading.Thread(target=record)
            thread.start()

        def on_stop_button_clicked(b):
            "stops the recording, runs Whisper, and forward the result to process_input"
            self.recording = False
            record_button.disabled=False
            stop_button.disabled=True
            audio_data = np.frombuffer(b"".join(self.frames), np.int16).astype(np.float32) * (1 / 32768.0)
            output = self.asr_engine.transcribe(audio_data)

            # We define the confidence score based on the log-probabilities
            conf_score = np.exp(np.mean([segment["avg_logprob"] for segment in output["segments"]]))
            # (and we push those up a bit, as the Whisper scores seem too low)
            conf_score = min(1, conf_score*1.2)

            # Finally, we process the input
            self.process_input(output["text"], conf_score)

        # The record and stop buttons
        record_button = widgets.Button(icon="microphone")
        stop_button = widgets.Button(icon="stop", disabled=True)
        record_button.on_click(on_record_button_clicked)
        stop_button.on_click(on_stop_button_clicked)
        
        # The chat area
        self.history_area = widgets.HTML(layout=widgets.Layout(width="600px", height="300px", 
                                                               border='1px solid black', overflow='scroll'))

        # The right side of the GUI
        right_side = widgets.VBox([widgets.Label(""), self.history_area, widgets.HBox([record_button, stop_button])])
        
        extended_gui = widgets.HBox([core_gui, right_side])
        return extended_gui


Let's give it a try:

In [ ]:
elevator = TalkingElevator()

Loading TTS and ASR models...

100%|███████████████████████████████████████| 461M/461M [00:15<00:00, 30.4MiB/s]


Done


We hope that the speech recognition and speech synthesis will work correctly -- if it isn't the case, do let us know ! (audio processing in Python can be quite tricky and will work differently from OS to OS). [^1]

[^1]: If you are running on Linux and the TTS is not working, install the following packages on your machine: `sudo apt update && sudo apt install espeak ffmpeg libespeak1`

**Note**: The current implementation reloads the TTS and ASR models every time, which means that you may run into a "CUDA: out of memory" error if you reinitialise the `TalkingElevator` many times. If this happens, simply restart the Python kernel, which will clear the memory on both the CPU and the GPU. 


### Intent recognition

We wish our talking elevator to support the following functionalities:
 
- If the user express a wish to go to floor $X$ (where $X$ is an integer value between 1 and 10), the elevator should go to that floor. The interface should allow for several ways to express a given intent, such as "_Please go to the $X$-th floor_" or "_Floor $X$, please_".
- The user requests can also be relative, for instance "_Go one floor up_".
- The elevator should provide _grounding_ feedback to the user. For instance, it should respond "_Ok, going to the $X$-th floor_" after a user request to move to $X$.  
- The elevator should handle misunderstandings and uncertainties, e.g. by requesting the user to repeat, or asking the user to confirm if the intent is uncertain (say, when its confidence score is lower than 0.5). 
- The elevator should also allow the user to ask where the office of a given employee is located. For instance, the user could ask "_where is Erik Velldal's office?_", and the elevator would provide a response such as "_The office of Erik Velldal is on the 4th floor. Do you wish to go there?_".  We provide you with the office numbers of a small set of IFI employees in the `OFFICES` dictionary (see below).
- The elevator should also be able to inform the user about the current floor (such as replying to "_Which floor are we on?_" or "_Are we on the 5th floor?_"). 
- Finally, if the user asks the elevator to stop (or if the user says "_no_" after a grounding feedback "_Ok, going to floor $X$._"), the elevator should stop, and ask for clarification regarding the actual user intent. 

To implement this conversational behaviour, we will rely on a classical NLU-based approach in which we will recognise the user _intent_, and then determine a response based on the recognised intent(s). 

__Task 1.1__ (1 point): You first need to define a list of user intents that cover the kinds of user inputs you expect to observe in this talking elevator, such as `RequestMoveToFloor` or `Confirm`. This is a design question, and there is no obvious right or wrong answer. Define below the intents you want to cover, along with an explanation and a few examples of user inputs for each.


<!-- Provide here the list of intent classes you have defined, together with an explanation and a few examples -->
**Answer:**

1. `RequestMoveToFloor`: This intent is used when the user wants to move to a specific floor. Examples: "Please go to the 5th floor", "Floor 3, please", "I want to go to the 7th floor".
2. `RequestMoveRelative`: This intent is used when the user wants to move to a floor relative to the current floor. Examples: "Go one floor up", "Move two floors down", "Take me to the floor above".
3. `RequestOfficeLocation`: This intent is used when the user wants to know the location of an employee's office. Examples: "Where is Erik's office?", "Can you tell me where the office of Erik is?", "I need to know where Erik's office is".
4. `RequestCurrentFloor`: This intent is used when the user wants to know the current floor. Examples: "Which floor are we on?", "Are we on the 5th floor?", "Can you tell me the current floor?".
5. `Confirm`: This intent is used when the user confirms the elevator's response. Examples: "Yes", "That's correct", "I confirm".
6. `Stop`: This intent is used when the user wants the elevator to stop. Examples: "Stop", "I want to stop", "Please stop".
7. `Repeat`: This intent is used when the user wants the elevator to repeat the last response. Examples: "Can you repeat that?", "I didn't hear you", "What did you say?".

__Task 1.2__ (1 points): We wish to build a classifier any user input to a probability distribution over those intents, and start by creating a small, synthetic training set. Make a list of about 100 user utterances, each labelled with an intent defined above. You can "make up" those utterances yourself, or ask someone else to come with alternative formulations if you lack inspiration.

In [7]:
labelled_utterances = [
    # Move Intent
    ("Go to floor 1", "Move"),
    ("Take me to floor 2", "Move"),
    ("Please go up to the 5th floor", "Move"),
    ("Take me to the top floor", "Move"),
    ("I’d like to go to floor 3", "Move"),
    ("Move to floor 4", "Move"),
    ("Take us to the first floor", "Move"),
    ("Bring me to the highest floor", "Move"),
    ("Could we go to floor 6?", "Move"),
    ("Can you take me to floor 8?", "Move"),
    ("Please move to floor 7", "Move"),
    ("I’d like to go down to the first floor", "Move"),
    ("Take us up to floor 10", "Move"),
    ("Let's go to floor 9", "Move"),
    ("Bring me to floor 2", "Move"),
    ("Elevator, please move to floor 3", "Move"),
    ("Take me down to the ground floor", "Move"),
    ("Take me up a floor", "Move"),
    ("Let's head to floor 5", "Move"),
    ("I want to go to floor 1", "Move"),
    ("Take me to the basement", "Move"),
    ("Could you take me down?", "Move"),
    ("Let’s head down", "Move"),
    ("Will you take me to the rooftop?", "Move"),
    ("I'd like to visit floor 7", "Move"),
    ("Could you stop at floor 3?", "Move"),
    ("Take me to the middle floor", "Move"),
    ("Bring me down to the lobby", "Move"),
    ("Take me down to floor 2", "Move"),
    ("Let's ride to floor 6", "Move"),
    ("Please go up to floor 4", "Move"),
    ("Take us down to the entrance", "Move"),
    ("Elevator, take me to floor 8", "Move"),
    ("Go up a floor, please", "Move"),
    ("Take us to floor 3", "Move"),
    ("Let's stop at floor 5", "Move"),
    ("Move to floor 10", "Move"),
    ("Let’s reach the top floor", "Move"),
    ("Let's proceed to floor 4", "Move"),
    ("Bring us to the lowest level", "Move"),
    ("I'd like to head up to floor 9", "Move"),
    ("Let’s go to floor 1", "Move"),
    ("Head to the second level", "Move"),

    # Stop Intent
    ("Can you stop here?", "Stop"),
    ("Stop the elevator", "Stop"),
    ("Stop moving", "Stop"),
    ("Stop at the next floor", "Stop"),
    ("Please stop", "Stop"),
    ("Stop the elevator at once", "Stop"),
    ("End the ride", "Stop"),
    ("Stop right here", "Stop"),
    ("Stop the lift", "Stop"),
    ("Can you wait here?", "Stop"),
    ("Stop immediately", "Stop"),
    ("Let’s halt here", "Stop"),
    ("Stop the elevator for me", "Stop"),
    ("Elevator, stop now", "Stop"),
    ("Can you pause here?", "Stop"),

    # Status Inquiry Intent
    ("What floor am I on?", "Status Inquiry"),
    ("Which floor are we at now?", "Status Inquiry"),
    ("Are we moving?", "Status Inquiry"),
    ("Is the elevator moving?", "Status Inquiry"),
    ("Tell me the current floor", "Status Inquiry"),
    ("Is this elevator moving up?", "Status Inquiry"),
    ("Where are we now?", "Status Inquiry"),
    ("Which floor are you at?", "Status Inquiry"),
    ("What’s our current location?", "Status Inquiry"),
    ("Are we on the main floor?", "Status Inquiry"),
    ("Where is the elevator now?", "Status Inquiry"),
    ("Are we on the way up?", "Status Inquiry"),
    ("What’s the highest floor here?", "Status Inquiry"),
    ("Where are we headed?", "Status Inquiry"),
    ("Are we there yet?", "Status Inquiry"),
    ("What floor are we passing?", "Status Inquiry"),
    ("Are we close to floor 7?", "Status Inquiry"),
    ("Which floor are you on?", "Status Inquiry"),
    ("Is this the top floor?", "Status Inquiry"),

    # Greeting/Exit Intent
    ("Hello", "Greeting/Exit"),
    ("Goodbye", "Greeting/Exit"),
    ("Hello there!", "Greeting/Exit"),
    ("Hi", "Greeting/Exit"),
    ("Bye", "Greeting/Exit"),
    ("Hey there", "Greeting/Exit"),
    ("Nice to meet you, elevator!", "Greeting/Exit"),
    ("Good afternoon", "Greeting/Exit"),

    # OutOfCoverage Intent
    ("I don’t need a ride", "OutOfCoverage"),
    ("Close the door", "OutOfCoverage"),
    ("Let's take the stairs", "OutOfCoverage"),
    ("How many floors are there?", "OutOfCoverage"),
    ("Are you on a break?", "OutOfCoverage"),
    ("I don't need the elevator", "OutOfCoverage"),
    ("Is the elevator moving fast?", "OutOfCoverage"),
    ("Are you stopping at every floor?", "OutOfCoverage"),
    ("Is anyone else here?", "OutOfCoverage"),
    ("I really like the IN4080 course", "OutOfCoverage")
]

We will now train an intent classifier based on the labelled utterances you have defined. To do so, we will rely on the [SetFit](https://huggingface.co/docs/setfit/index) library, which allows one to easily train a text classification model from few examples by fine-tuning a sentence-transformer model (like the ones we used in oblig 2 and 3). Make sure that the `setfit` library is installed (`pip install setfit`).

Read the [Setfit quickstart guide](https://huggingface.co/docs/setfit/quickstart) to find out how to use the library.

__Task 1.3__ (2 points): Implement the `__init__`, `train` and `get_intent_distrib` methods of the `IntentClassifier` class below. The classifier should rely on a `Setfit` model trained on the labelled utterances you have already defined. 

In [19]:
import setfit, datasets

class IntentClassifier:

    def __init__(self, model_name="sentence-transformers/paraphrase-mpnet-base-v2"):
        """Initialises the setfit model that will be used for the intent recognition"""

        raise NotImplementedError("Must be implemented")
    
    def train(self, labelled_utterances: List[Tuple[str,str]]):
        """Trains the setfit model on the labelled utterances"""

        # Creates the dataset from the list of labelled utterances
        train_data = datasets.Dataset.from_list([{"text":utt, "label":label} 
                                                 for utt,label in labelled_utterances])
        
        raise NotImplementedError("Must be implemented")
    
    def get_intent_distrib(self, utterance:str):
        """Applies the trained model on a new utterance. The method should return a
        dictionary that maps each possible intent category to a probability."""

        raise NotImplementedError("Must be implemented")


In [ ]:
classifier = IntentClassifier()
classifier.train(labelled_utterances)

Since we don't have any test data, we cannot really conduct an evaluation of the classification performance, but this step would be of course strongly adviced when developing a real system. 

### Slot filling

In addition to the intents themselves, we also wish to detect some slots, such as floor numbers or person names. For this step, we will not use a data-driven model, but rather rely on an old-fashioned, rule-based approach:
- For floor numbers, we will rely on string matching (with regular expressions or basic string search) that detect patterns such as "X floor" (where X is [first,second, third, fourth, fifth, sixth, seventh, eighth, ninth, tenth]) or "floor X" (where X is between 1 and 10).
- For person names, we have a predefined list of person names to detect (employees at IFI), and we should simply search for their occurrence in the user input. The simplest implementation is to just for look for exact occurrences. However, since speech recognition will often struggle to recognize foreign person names, an even better approach would be to search for names that are phonetically close (you can use the `jellyfish` library for this).

The results of the slot filling should be a dictionary mapping slot names to a canonical form of the slot value. For instance, if the utterance contains the expression "ninth floor", the resulting slot dictionary should be `{"floor_number":9}`. Similarly, the `employee_name` slot should be a name present in `OFFICES` dictionary. 

__Task 1.4__ (2 points): Implement the method `fill_slots` that will detect the occurrence of those slots in the user input.<br>
(+ 1 bonus point if you implement a fuzzy matching strategy to find person names that are phonetically close)

In [21]:

# Floor numbers for a subset of the IFI employees
OFFICES = {'Adín Ramírez Rivera': 4, 'Andreas Austeng': 4, 'Anne H Schistad Solberg': 4, 
           'Arild Torolv Søetorp Waaler': 9, 'Audun Jøsang': 9, 'Birthe Soppe': 4, 'Carsten Griwodz': 4,
           'Dag Sjøberg': 9, 'Dag Trygve Eckhoff Wisland': 5, 'Einar Broch Johnsen': 8, 
           'Eric Bartley Jul': 10, 'Erik Velldal': 4, 'Henrik Skaug Sætra': 7, 'Ingrid Chieh Yu': 8,
           'Jørn Anders Braa': 6, 'Kristin Bråthen': 4, 'Kyrre Glette': 4, 'Lars Groth': 6, 
           'Lilja Øvrelid': 4, 'Maja Van Der Velden': 7, 'Martin Giese': 9, 'Michael Welzl': 5, 
           'Miria Grisot': 6, 'Nils Gruschka': 9, 'Olaf Owe': 9, 'Ole Christian Lingjærde': 4, 
           'Ole Hanseth': 6, 'Paulo Ferreira': 10, 'Philipp Dominik Häfliger': 5, 'Philipp Häfliger': 5, 
           'Roman Vitenberg': 4, 'Silvia Lizeth Tapia Tarifa': 8, 'Stephan Oepen': 4, 
           'Sundeep Sahay': 6, 'Thomas Peter Plagemann': 4, 'Tone Bratteteig': 7, 'Torbjørn Rognes': 8, 
           'Truls Erikson': 6, 'Viktoria Stray': 10, 'Yngvar Berg': 5, 'Yves Scherrer': 4, 
           'Özgü Mira Alay-Erduran': 4}

def fill_slots(user_input:str) -> Dict[str,str]:
    """Extracts the set of slots detected in the user inputs. More precisely, the method
    should detect both floor numbers and person names, and return a dictionary mapping slot 
    names (in this case either `floor_number` or `employee_name`) to its corresponding
    value, in canonical form (integer for the floor number, string for the employee name)"""

    raise NotImplementedError()


### Response selection

The next step is to implement the response selection mechanism. The response will depend on various factors:
- the inferred user intents from the user utterance
- the detected slot values in the user utterance (if any)
- the current floor
- the list of next floor stops that are yet to be reached
- the dialogue history (as a list of dialogue turns).

The response may consist of verbal responses (enacted by calls to `_say_to_user`) but also physical actions, represented by calls to either `move_to_floor` or `stop`. 

__Task 1.5__ (3 points): Implement the method `_respond`, which is responsible for selecting and executing those responses. The responses should satisfy the aforementioned conversational criteria (provide grounding feedback, use confirmations and clarification requests etc.). This method will consist in practice of many _if...then...else_ blocks. 

In [ ]:
def _respond(self, intent_distrib: Dict[str, float], slots: Dict[str,str]) :
    """Given a probability distribution over possible intents, an a (possibly empty) list
    of detected slots in the user input, decide how to react. The method should lead
    to calls to both physical actions (move_to_floor, stop) and dialogue responses 
    (via _say_to_user)."""

    raise NotImplementedError("")

setattr(TalkingElevator, "_respond", _respond)



### Putting it all together

The last step is to implement the `process_input` method in the `TalkingElevator` class. The method should rely on the intent recognition, slot filling and response selection mechanism (which you have implemented in the previous steps) to react to a given user input.

**Task 1.6** (1 point): Implement the `process_input` method:

In [ ]:
    

def process_input(self, user_input: str, conf_score:float=1.0):
    """Processes the (transcribed) user input, and respond appropriately 
    (through a verbal response and possibly also an action, such as moving floors).
    The method should rely on the intent classifier, slot-filling function, and
    response selection function."""

    self._add_to_dialogue_history(user_input, speaker="user", conf_score=conf_score)
    raise NotImplementedError()

setattr(TalkingElevator, "process_input", process_input)

We are now ready to test our talking elevator: 


In [ ]:
elevator = TalkingElevator()

Your talking elevator will mostly likely not function properly right from the start. Identify what works and what doesn't and correct the code you have developed in Tasks 1.1 - 1.6 until your system meets the specifications we have outlined. 

## Part 2 : Machine translation

In this part, we evaluate a pre-trained machine translation model on data from the Lord of the Rings movies and fine-tune it to improve the translation quality.

### Data

We provide you with two files, `lotr.detok.de` and `lotr.detok.en`, containing German and English movie subtitles. These two files constitute a so-called _parallel corpus_, i.e. each sentence/line in German corresponds to a sentence/line in English. The two files have the same number of lines and the German sentence on line $i$ corresponds to the English sentence on line $i$. The subtitles are extracted from the [OpenSubtitles-2018](https://opus.nlpl.eu/OpenSubtitles/corpus/version/OpenSubtitles) corpus.

Here are the first ten lines of the two files:

<style scoped>
table {
  font-size: 12px;
}
</style>
| Nb  | German (`lotr.detok.de`)         | English (`lotr.detok.en`)      |
|---|----------------------------------|--------------------------------|
| 1 | Die Welt ist im Wandel. | The world is changed.   |
| 2 | Ich spüre es im Wasser. | I feel it in the water. |
| 3 | Ich spüre es in der Erde. | I feel it in the earth. |
| 4 | Ich rieche es in der Luft. | I smell it in the air. |
| 5 | Vieles, was einst war, ist verloren, da niemand mehr lebt, der sich erinnert. | Much that once was is lost. For none now live who remember it. |
| 6 | Es begann mit dem Schmieden der Großen Ringe. | It began with the forging of the Great Rings. |
| 7 | 3 wurden den Elben gegeben, den unsterblichen, weisesten und reinsten aller Wesen. | Three were given to the Elves: Immortal, wisest and fairest of all beings. |
| 8 | 7 den Zwergenherrschern, großen Bergleuten und Handwerkern in ihren Hallen aus Stein. | Seven to the Dwarf-lords: Great miners and craftsmen of the mountain halls. |
| 9 | Und 9... 9 Ringe wurden den Menschen geschenkt, die vor allem anderen nach Macht streben. | And nine nine rings were gifted to the race of Men who, above all else, desire power. |
| 10 | Denn diese Ringe bargen die Kraft und den Willen, jedes Volk zu leiten. | For within these rings was bound the strength and will to govern each race. |


### Getting started

We will a pretrained machine translation model for German-to-English translation. The model is available on the HuggingFace model hub and can be used with the `transformers` library.

Let us first make sure that all required modules are installed:

In [ ]:
%pip install torch transformers accelerate evaluate sacrebleu sacremoses sentencepiece unbabel-comet

The bilingual model is called [`opus-mt-de-en`](https://huggingface.co/Helsinki-NLP/opus-mt-de-en) and has been trained by the Helsinki-NLP group. Like (almost) all HuggingFace models, it consists of a _tokenizer_ and the _sequence-to-sequence model_ properly speaking. We need to load both separately:

In [ ]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained("helsinki-nlp/opus-mt-de-en")
translator = transformers.AutoModelForSeq2SeqLM.from_pretrained("helsinki-nlp/opus-mt-de-en")

# Change "cuda" to "cpu" if you're running on a machine without GPU
device = "cuda"
translator = translator.to(device)

The `transformers` library will automatically download the models from the HuggingFace hub the first time you run this cell, so it may take a bit longer.

Let's take the first two German sentences, tokenize them, and translate them to English:

In [ ]:
tokens = tokenizer(["Die Welt ist im Wandel.", "Ich spüre es im Wasser."], return_tensors="pt", padding=True)
print(tokens)

In [ ]:
outputs = translator.generate(**tokens.to(device), max_new_tokens=50)
print(outputs)

__Task 2.1__ (1 point):
- What do the numbers in the `input_ids` represent?
- What is the effect of `padding=True`? How would the data look like if padding was disabled?
- What does `max_new_tokens` do? Why do you think it is important to set this parameter?

We can get actual words by running the output through the `batch_decode` function of the tokenizer:

In [ ]:
translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(translations)

__Note:__ We assume that you will run the translations from German to English. If you would like to work on the opposite translation direction (and feel comfortable evaluating the German output), you are welcome to do so. The corresponding bilingual model is called `opus-mt-en-de`.

### Data splitting

Before we move on, we need to split our data. We will evaluate different models and for that we'll need test data. We will also fine-tune a model, and for that we'll need training data. The entire Lord of the Rings dataset has 9640 lines.

__Task 2.2__ (1 point): Split the dataset in such a way that the **last** 1000 lines are used for testing and the remaining lines (8640) for training. Save the data under the following filenames: `lotr.train.de, lotr.train.en, lotr.test.de, lotr.test.en`. You can use Python code or other tools to perform the splitting.

In [ ]:
# code here

__Task 2.3__ (1 point): What are potential risks and drawbacks of splitting the dataset in this way? 


Now we are ready to translate the test set with our model.

__Task 2.4__ (2 points): Create a function that loads the entire `lotr.test.de` file, translates each line with the `opus-mt-de-en` model and writes its output to a new file, one sentence per line.

The easiest way to do this is to just load the entire test file into a list, tokenize and translate it, but the test set may be too large to fit on GPU memory, or it might be inefficient and slow if you use a CPU. A better alternative is to split the data into batches of 50-100 sentences and send each batch separately to the translator.

In [ ]:
def translate(input_file, translation_file, tokenizer, translator, batch_size=100):
    """Translate an input file line by line using the loaded tokenizer and translator,
    and write the translations to output_file."""
    pass


translate("lotr.test.de", "lotr.output_opus.en", tokenizer, translator)

Before moving on, open the output file and check that the translations look ok. In particular, the file should contain the expected number of lines and output should be in the expected language (English or German, depending on the chosen direction).

__Task 2.5__ (1 point): Open both the output file and the reference translations (`lotr.test.en` if translating from German to English) and compare the first 20 lines. How would you rate the translations of the OPUS system on a scale from 1 (incomprehensible and/or completely different meaning) to 5 (grammatically correct and meaning fully preserved)? Justify your answer.

### Evaluation

We can now evaluate the quality of our translations. In a first step, we perform _reference-based surface-level evaluation_  using the popular BLEU score. We can do that with the `sacrebleu` module. Below is a slightly reformatted example taken from the [SacreBLEU documentation](https://github.com/mjpost/sacrebleu/tree/master?tab=readme-ov-file#using-sacrebleu-from-python):

In [ ]:
from sacrebleu.metrics import BLEU

reference = ['The dog bit the man.', 'It was not unexpected.', 'The man bit him first.']
hypothesis = ['The dog bit the man.', "It wasn't surprising.", 'The man had just bitten him.']

bleu_scorer = BLEU()
# BLEU can deal with multiple references per sentence, but here we only have one, so we just enclose it in another set of brackets:
score = bleu_scorer.corpus_score(hypothesis, [reference])
print(score)

__Task 2.6__ (1 point): Load both the system output and the reference of your test set and compute the corpus-level BLEU score. Also compute the corpus-level chrF score. Which of the scores is higher?

In [ ]:
from sacrebleu.metrics import BLEU, CHRF

def evaluate_bleu(hypothesis_file, reference_file):
	pass

evaluate_bleu("lotr.output_opus.en", "lotr.test.en")

Besides string-based metrics, neural metrics have become increasingly popular lately, since they have been shown to correlate better with human judgements. The most popular neural metric is called COMET and it can be used with the HuggingFace `evaluate` package. The example below is from the [documentation](https://huggingface.co/spaces/evaluate-metric/comet/blob/main/README.md):

In [ ]:
import evaluate

comet_metric = evaluate.load('comet')
src = ["Dem Feuer konnte Einhalt geboten werden", "Schulen und Kindergärten wurden eröffnet."]
hyp = ["The fire could be stopped", "Schools and kindergartens were open"]
ref = ["They were able to control the fire.", "Schools and kindergartens opened"]
comet_score = comet_metric.compute(predictions=hyp, references=ref, sources=src)
print(comet_score)

__Task 2.7__ (1 point): Adapt this code to evaluate the output of the OPUS model. Note that COMET also requires the source text.

In [ ]:
def evaluate_comet(hypothesis_file, reference_file, source_file):
    pass


### Fine-tuning

Let us see now if we can further improve the translation quality. We still haven't used the training set after all...

Fine-tuning a translation model with the `transformers` library is a bit convoluted. You need the following ingredients:
- A `Seq2SeqTrainer` object, which defines the initial model and its tokenizer, the training data, and the configuration parameters (as a `Seq2SeqTrainingArguments` object). The training process starts with the `train()` method.
- A `Seq2SeqTrainingArguments` object, which contains the configuration parameters, such as the number of training epochs, the path for saving the fine-tuned model, the learning rate etc.
- A `DataCollatorForSeq2Seq` object that takes care of splitting the training data into batches of appropriate size.
- A `DatasetDict` object containing the tokenized training data. Typically, the untokenized data is loaded into a `DatasetDict` object, and the tokenization function is applied to everything inside this `DatasetDict` using the `map()` function.

__Task 2.8__ (1 point): The code in the box below shows a working example using the pretrained OPUS model, but is limited to two sentence pairs. Complete the code to load the entire training data.

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict

max_length = 100

ds = Dataset.from_dict({
    "src_text": ["Die Welt ist im Wandel.", "Ich spüre es im Wasser."],
    "tgt_text": ["The world is changed.", "I feel it in the water."]
})
data = DatasetDict({"train": ds})

def preprocess_function(examples):
    model_inputs = tokenizer(examples["src_text"], text_target=examples["tgt_text"], max_length=max_length, truncation=True)
    return model_inputs

tokenized_datasets = data.map(preprocess_function, batched=True)
print(tokenized_datasets)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=translator)

args = Seq2SeqTrainingArguments(
    output_dir="opus-mt-de-en-lotr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    translator,
    args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

The model was fine-tuned for three epochs, and you should have three checkpoints in the `opus-mt-de-en-lotr` directory.

__Task 2.9__ (1 point): Choose one of the checkpoints and use it to translate the test set. Evaluate the test set with BLEU, chrF and COMET. Note that locally saved model files (and tokenizers) can be loaded in the same way as models from the HuggingFace hub, e.g. with the following command: `transformers.AutoModelForSeq2SeqLM.from_pretrained("opus-mt-de-en-lotr/checkpoint-810")`

Did fine-tuning help? Did fine-tuning help? Have a look at the first rows of the files. Do you agree with the metrics?

In [ ]:
# code here